In [2]:
import sys

sys.path.append('../')

import time
from canopus import *
from canopus.utils import *
from qiskit.transpiler import CouplingMap, PassManager
from pytket.utils import compare_unitaries
from qiskit import qasm2

from qiskit.circuit.library import QFT

# configure logging
# logging.basicConfig(
#     level=logging.INFO,
#     format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
#     handlers=[logging.StreamHandler()]
# )


qc = qasm2.loads("""
OPENQASM 2.0;
include "qelib1.inc";
qreg q[6];
h q[0];
h q[2];
h q[5];
z q[0];
cx q[1],q[2];
cx q[4],q[5];
cx q[0],q[1];
cx q[2],q[3];
h q[2];
h q[3];
cx q[1],q[2];
cx q[3],q[5];
z q[3];
cx q[3],q[4];
cx q[0],q[3];
""")

circ = qiskit_to_tket(qc)
circ = rebase_to_tk2(circ)
qc = tket_to_qiskit(circ)

console.rule('Original circuit')

# print(qc)
assert compare_unitaries(circ.get_unitary(), circ2mat(qc))

print(qiskit_to_tket(qc).get_commands())

coupling_map = CouplingMap.from_line(num_qubits=qc.num_qubits)
# coupling_map = CouplingMap.from_grid(np.ceil(np.sqrt(qc.num_qubits)).astype(int), np.ceil(np.sqrt(qc.num_qubits)).astype(int))
backend = CanopusBackend(coupling_map, 'cx', 'xy')

console.print('Pulse duration: {:.4f}'.format(backend.cost_estimator.eval_circuit_duration(qc)))

console.rule('SABRE mapping')
start = time.perf_counter()
pm = PassManager(SabreMapping(backend, seed=123))
qc_sabre = pm.run(qc)
end = time.perf_counter()
# print(qc_sabre)
qc_sabre_opt = tket_to_qiskit(rebase_to_tk2(qiskit_to_tket(qc_sabre)))
# print(qc_sabre_opt)
console.print('Pulse duration: {:.4f}'.format(backend.cost_estimator.eval_circuit_duration(qc_sabre_opt)))
console.print('Time taken for Canopus mapping: {:.4f} seconds'.format(end - start))

console.rule('Canopus mapping')
start = time.perf_counter()
pm = PassManager(CanopusMapping(backend, seed=123))
qc_canopus = pm.run(qc)
end = time.perf_counter()
# print(qc_canopus)
qc_canopus_opt = tket_to_qiskit(rebase_to_tk2(qiskit_to_tket(qc_canopus)))
# print(qc_canopus_opt)
console.print('Pulse duration: {:.4f}'.format(backend.cost_estimator.eval_circuit_duration(qc_canopus_opt)))
console.print('Time taken for Canopus mapping: {:.4f} seconds'.format(end - start))


──────────────────────────────────────────────── Original circuit ─────────────────────────────────────────────────

[U3(3.5, 1, 0.5) q[1];, U3(1.5, 0, 0.5) q[2];, U3(1.5, 1.5, 1.5) q[3];, U3(3.5, 1, 0.5) q[4];, U3(1.5, 0, 1) q[5];, can(0.5/pi,0,0) q[1], q[2];, can(0.5/pi,0,0) q[4], q[5];, U3(1.5, 0, 0.5) q[1];, U3(0.5, 0, 0.5) q[2];, U3(1.5, 0, 0.5) q[4];, can(0.5/pi,0,0) q[0], q[1];, can(0.5/pi,0,0) q[2], q[3];, U3(0.5, 0, 0.5) q[1];, U3(1.5, 1.5, 1.5) q[2];, U3(0, 1.5, 1.5) q[3];, can(0.5/pi,0,0) q[1], q[2];, can(0.5/pi,0,0) q[3], q[5];, U3(0.5, 0, 1) q[1];, U3(0, 1.5, 1.5) q[2];, can(0.5/pi,0,0) q[3], q[4];, U3(0, 1.5, 1.5) q[5];, U3(1.5, 0, 0.5) q[3];, U3(0, 1.5, 1.5) q[4];, can(0.5/pi,0,0) q[0], q[3];, U3(0.5, 0, 1) q[0];, U3(0, 1.5, 1.5) q[3];]


Pulse duration: 5.0000

────────────────────────────────────────────────── SABRE mapping ──────────────────────────────────────────────────

Pulse duration: 11.0000

Time taken for Canopus mapping: 0.0887 seconds

───────────────────────────────────────────────── Canopus mapping ─────────────────────────────────────────────────

Pulse duration: 10.0000

Time taken for Canopus mapping: 0.0923 seconds